In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
import math
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

%matplotlib inline
sns.set_style('white')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv')

In [3]:
df.head()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NEW YORK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Offenses Known to Law Enforcement,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"by City, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape\n(revised\ndefinition)1,Rape\n(legacy\ndefinition)2,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson3
4,Adams Village,"1,861",0,0,NaN,0,0,0,12,2,10,0,0


In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv', skiprows=4)

In [5]:
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,nan,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,nan,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,nan,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,nan,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,nan,3,4,16,223,53,165,5,nan


In [6]:
df = df.drop('Rape\n(revised\ndefinition)1', axis=1)

In [7]:
df.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,"1,861",0,0.000,0,0,0,12,2,10,0,0.000
1,Addison Town and Village,"2,577",3,0.000,0,0,3,24,3,20,1,0.000
2,Akron Village,"2,846",3,0.000,0,0,3,16,1,15,0,0.000
3,Albany,"97,956",791,8.000,30,227,526,"4,090",705,"3,243",142,nan
4,Albion Village,"6,388",23,0.000,3,4,16,223,53,165,5,nan


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 12 columns):
City                                    351 non-null object
Population                              348 non-null object
Violent
crime                           348 non-null object
Murder and
nonnegligent
manslaughter    348 non-null float64
Rape
(legacy
definition)2               348 non-null object
Robbery                                 348 non-null object
Aggravated
assault                      348 non-null object
Property
crime                          348 non-null object
Burglary                                348 non-null object
Larceny-
theft                          348 non-null object
Motor
vehicle
theft                     348 non-null object
Arson3                                  187 non-null float64
dtypes: float64(2), object(10)
memory usage: 19.2+ KB


In [9]:
df.columns = ['city', 'population', 'violent_crime', 'murder', 'rape_legacy', 'robbery', 'aggravated_assault', 'property_crime', 'burglary', 'larceny_theft', 'vehicle_theft', 'arson']
print(df.head())

                       city population violent_crime  murder rape_legacy  \
0             Adams Village      1,861             0   0.000           0   
1  Addison Town and Village      2,577             3   0.000           0   
2             Akron Village      2,846             3   0.000           0   
3                    Albany     97,956           791   8.000          30   
4            Albion Village      6,388            23   0.000           3   

  robbery aggravated_assault property_crime burglary larceny_theft  \
0       0                  0             12        2            10   
1       0                  3             24        3            20   
2       0                  3             16        1            15   
3     227                526          4,090      705         3,243   
4       4                 16            223       53           165   

  vehicle_theft  arson  
0             0  0.000  
1             1  0.000  
2             0  0.000  
3           142    nan

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 12 columns):
city                  351 non-null object
population            348 non-null object
violent_crime         348 non-null object
murder                348 non-null float64
rape_legacy           348 non-null object
robbery               348 non-null object
aggravated_assault    348 non-null object
property_crime        348 non-null object
burglary              348 non-null object
larceny_theft         348 non-null object
vehicle_theft         348 non-null object
arson                 187 non-null float64
dtypes: float64(2), object(10)
memory usage: 19.2+ KB


In [18]:
df = df.dropna()
column_names = df.columns

for column in column_names:
    df[column] = df[column].astype('str').str.replace(',', '')
    df[column] = df[['burglary','larceny_theft','population','violent_crime','murder','rape_legacy','robbery','aggravated_assault','property_crime','vehicle_theft', 'arson']].astype('float')

regr = linear_model.LinearRegression()
Y = df['property_crime'].astype('float')
X = df[['burglary','larceny_theft']].astype('float')
Y = Y.values.reshape(-1, 1)
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[0.5 0.5]]

Intercept: 
 [-2.84217094e-14]

R-squared:
1.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 346
Data columns (total 12 columns):
city                  187 non-null float64
population            187 non-null float64
violent_crime         187 non-null float64
murder                187 non-null float64
rape_legacy           187 non-null float64
robbery               187 non-null float64
aggravated_assault    187 non-null float64
property_crime        187 non-null float64
burglary              187 non-null float64
larceny_theft         187 non-null float64
vehicle_theft         187 non-null float64
arson                 187 non-null float64
dtypes: float64(12)
memory usage: 19.0 KB


In [24]:
linear_formula = 'property_crime ~ murder + robbery'
lm = smf.ols(formula=linear_formula, data=df).fit()

In [25]:
lm.params

Intercept   0.000
murder      0.500
robbery     0.500
dtype: float64

In [26]:
lm.pvalues

Intercept   0.022
murder      0.000
robbery     0.000
dtype: float64

In [27]:
lm.rsquared

1.0